In [ ]:
# Import Packages
import torch
import torchvision
from torchvision import models, transforms
from torchvision.datasets import ImageFolder
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Subset, DataLoader, random_split
import numpy as np
import os
import shutil
from torchsummary import summary
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import label_binarize
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_curve, auc
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
# Image Transformations for AlexNet
transform = transforms.Compose([
    transforms.Resize(224), # se ajusta la imagen a 224x224 pixeles
    transforms.CenterCrop(224), # se corto del centro a 224
    transforms.ToTensor(), # Convierte a imagen PIL (Python Imaging Library) en un tensor de PyTorch
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # se normalizan los valores estandar RGB
])

In [ ]:
# Assigning a Class Based on the Folder Name"

# Data
data = 'C:/Users/itz_l/Desktop/luzAro/datasets/'
# Create an ImageFolder for Your Dataset
# Automatically reads the folders and considers that the folder name corresponds to the class
dataset = ImageFolder(root=data, transform=transform)

# Get Image Indices for Each Class"
# I try to ensure that the same number of images is used for training
# This way, I ensure that all classes are included and none are left out
indices = {label: np.where(np.array(dataset.targets) == label)[0] for label in range(len(dataset.classes))}

# Dividir los índices para cada clase en entrenamiento y prueba
train_indices = []
test_indices = []
for label, idx in indices.items():
    np.random.shuffle(idx)
    train_size = int(0.9 * len(idx))
    train_indices.extend(idx[:train_size])
    test_indices.extend(idx[train_size:])

# Create Training and Testing Subsets with the Obtained Indices
train_data = Subset(dataset, train_indices)
test_data = Subset(dataset, test_indices)

# Create DataLoaders for the Training and Testing Sets
train_loader = DataLoader(train_data, batch_size=50, shuffle=True)
test_loader = DataLoader(test_data, batch_size=50, shuffle=False) 

# Check the Number of Samples in Each Set
print(f"Training Samples: {len(train_data)}")
print(f"Validation Samples: {len(test_data)}")

In [ ]:
# Path to the Cache Directory
cache_dir = os.path.expanduser('~/.cache/torch/hub/checkpoints')

# Delete the Cache Directory if It Exists
if os.path.exists(cache_dir):
    shutil.rmtree(cache_dir)
    print(f"Cache Directory Deleted: {cache_dir}")
else:
    print(f"Cache Directory Not Found at: {cache_dir}")
# Descargar modelo preentrenado AlexNet
alexnet = models.alexnet(pretrained=True)

In [ ]:
# Replace the Final Layer with a New Classifier
alexnet.classifier[6] = nn.Sequential(
    nn.Linear(alexnet.classifier[6].in_features, num_classes)
)

In [ ]:
# Function to calculate metrics
def calculate_metrics(all_labels, all_preds):
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='weighted')
    recall = recall_score(all_labels, all_preds, average='weighted')
    f1 = f1_score(all_labels, all_preds, average='weighted')
    conf_matrix = confusion_matrix(all_labels, all_preds)
    return accuracy, precision, recall, f1, conf_matrix

# Function to Calculate Probabilities, Predictions, and True Labels
def calculate_probs_and_preds(model, dataloader, device):
    all_probs = []
    all_preds = []
    all_labels = []
    model.eval()
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            probs = torch.softmax(outputs, dim=1)  # Probabilities by class
            _, predicted = torch.max(outputs, 1)   # Prediction class
            
            # Convert to NumPy Arrays and Extend Lists
            probs_np = probs.cpu().numpy()
            preds_np = predicted.cpu().numpy()
            all_probs.extend(probs_np)
            all_preds.extend(preds_np)
            
            # Capturar etiquetas verdaderas
            all_labels.extend(labels.cpu().numpy())
    
    return np.array(all_probs), np.array(all_preds), np.array(all_labels)

# Define the Model, Optimizer, and Loss Function
num_epochs = 50
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
alexnet = alexnet.to(device)
optimizer = optim.Adam(alexnet.parameters(), lr=0.000005)


criterion = nn.CrossEntropyLoss()

train_acc_history = []
val_acc_history = []
train_loss_history = []
val_loss_history = []

# Entrenamiento del modelo
for epoch in range(num_epochs):
    alexnet.train()  # Modo de entrenamiento
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    all_preds_train = []
    all_labels_train = []

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = alexnet(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

        _, predicted = torch.max(outputs, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

        # Collect predictions and labels for training phase metrics
        all_preds_train.extend(predicted.cpu().numpy())
        all_labels_train.extend(labels.cpu().numpy())

    epoch_loss = running_loss / len(train_loader.dataset)
    epoch_acc = correct_train / total_train
    train_loss_history.append(epoch_loss)
    train_acc_history.append(epoch_acc)

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}")

    # Calculate Metrics for the Training Phase
    accuracy_train, precision_train, recall_train, f1_train, _ = calculate_metrics(all_labels_train, all_preds_train)
    print(f"Training Metrics: Accuracy: {accuracy_train:.4f}, Precision: {precision_train:.4f}, Recall: {recall_train:.4f}, F1-score: {f1_train:.4f}")

    # Model Evaluation on the Validation Set
    alexnet.eval()
    running_val_loss = 0.0
    correct = 0
    total = 0

    all_preds_val = []
    all_labels_val = []
    all_probs_val = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = alexnet(inputs)
            probs = torch.softmax(outputs, dim=1)  # Probabilities by class
            _, predicted = torch.max(outputs, 1)
            
            # Add Probabilities and Predictions to the Validation Set
            all_probs_val.extend(probs.cpu().numpy())
            all_preds_val.extend(predicted.cpu().numpy())
            all_labels_val.extend(labels.cpu().numpy())
            
            loss = criterion(outputs, labels)
            running_val_loss += loss.item() * inputs.size(0)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    epoch_val_loss = running_val_loss / len(test_loader.dataset)
    epoch_val_acc = correct / total
    val_loss_history.append(epoch_val_loss)
    val_acc_history.append(epoch_val_acc)
    print(f"Validation Loss: {epoch_val_loss:.4f}, Validation Accuracy: {epoch_val_acc:.4f}")

    # Metrics calculation to each validation phaase
    accuracy_val, precision_val, recall_val, f1_val, _ = calculate_metrics(all_labels_val, all_preds_val)
    print(f"Validation Metrics: Accuracy: {accuracy_val:.4f}, Precision: {precision_val:.4f}, Recall: {recall_val:.4f}, F1-score: {f1_val:.4f}")

# OVerall metrics calculation
accuracy, precision, recall, f1, conf_matrix = calculate_metrics(all_labels_val, all_preds_val)
print(f"\nOverall Metrics on Validation Set:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")
print("\nConfusion Matrix:")
print(conf_matrix)


In [ ]:
# Convert all_probs_val to a NumPy array if it isn't already
all_probs_val = np.array(all_probs_val)

# Obtain Binary Labels for Each Class
y_true = np.array(all_labels_val)
y_score = np.array(all_probs_val)

# Class names
class_names = ["Class: 'High'", "Class: 'Medium'", "Class: 'Low'"]

# Calculate the ROC Curve and Area Under the Curve (AUC) for Each Class
fpr = dict()
tpr = dict()
roc_auc = dict()
num_classes = len(class_names)  # Número de clases

# Binarize the True Labels
y_true_bin = label_binarize(y_true, classes=range(num_classes))

for i in range(num_classes):
    # ROC for each class
    fpr[i], tpr[i], _ = roc_curve(y_true_bin[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Calculate the overall ROC curve for all classes"
# Interpolate FPR and TPR to ensure each class has the same number of points"
all_fpr = np.linspace(0, 1, 100)
mean_tpr = np.zeros_like(all_fpr)

for i in range(num_classes):
    mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])

mean_tpr /= num_classes
mean_auc = auc(all_fpr, mean_tpr)

# Crear figura y subplots
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Graficar las curvas ROC de cada clase en el primer subplot
colors = ['red', 'orange', 'green']  # Colores para las curvas ROC de cada clase
markers = ['o', 'x', '+']  # Marcadores para las curvas ROC de cada clase
linestyles = ['-', '-', '-']  # Estilos de línea para las curvas ROC de cada clase
linewidths = [2, 1.5, 1]  # Grosor de las líneas para cada clase

for i, color, marker, linestyle, lw in zip(range(num_classes), colors, markers, linestyles, linewidths):
    axes[0].plot(fpr[i], tpr[i], linestyle=linestyle, marker=marker, color=color, 
                 linewidth=lw, label=f'{class_names[i]} (AUC = {roc_auc[i]:.4f})')

axes[0].plot([0, 1], [0, 1], color='navy', lw=1, linestyle='--')
axes[0].set_xlim([0.0, 1.0])
axes[0].set_ylim([0.0, 1.05])
axes[0].set_xlabel('False Positive Rate', fontsize=17)
axes[0].set_ylabel('True Positive Rate', fontsize=17)
axes[0].set_title('')
axes[0].legend(loc="lower right",  fontsize='large')
axes[0].text(0.5, -0.2, '(a)', ha='center', va='center', transform=axes[0].transAxes, fontsize=17)

# Graficar la curva ROC promedio en el segundo subplot
axes[1].plot(all_fpr, mean_tpr, color='black', linestyle='--', lw=2, label=f'Overall (AUC = {mean_auc:.4f})')
axes[1].plot([0, 1], [0, 1], color='navy', lw=1, linestyle='--')
axes[1].set_xlim([0.0, 1.0])
axes[1].set_ylim([0.0, 1.05])
axes[1].set_xlabel('False Positive Rate', fontsize=17)
axes[1].set_ylabel('True Positive Rate', fontsize=17)
axes[1].set_title('')
axes[1].legend(loc="lower right",  fontsize='large')
axes[1].text(0.5, -0.2, '(b)', ha='center', va='center', transform=axes[1].transAxes, fontsize=17)
# Mostrar la figura con los subplots
plt.tight_layout()
plt.show()

In [ ]:
# Class names
class_names = ["High", "Medium", "Low"]

# Create figure
fig, axes = plt.subplots(1, 2, figsize=(18, 10))

# Normalized Confution matrix
sns.heatmap(conf_matrix, annot=True, cmap="Greens", fmt="d", cbar=False, ax=axes[1], 
            linewidths=1, xticklabels=class_names, yticklabels=class_names, annot_kws={"size": 18})
axes[1].set_title('', fontsize=25)
axes[1].set_xlabel('Predicted label', fontsize=22)
axes[1].set_ylabel('True label', fontsize=22)
axes[1].tick_params(axis='both', labelsize=18)

# Unnormalized Confution matrix
sns.heatmap(norm_conf_matrix, annot=True, cmap="Greens", fmt=".2f", cbar=False, ax=axes[0], 
            linewidths=1, xticklabels=class_names, yticklabels=class_names, annot_kws={"size": 20})
axes[0].set_title('', fontsize=25)
axes[0].set_xlabel('Predicted label', fontsize=22)
axes[0].set_ylabel('True label', fontsize=22)
axes[0].tick_params(axis='both', labelsize=18)  
fig.text(0.27, 0.08, '(a)', fontsize=22, ha='center')
fig.text(0.77, 0.08, '(b)', fontsize=22, ha='center')
plt.tight_layout(rect=[0, 0.1, 1, 1])  
plt.show()

In [ ]:

# Setup style
sns.set(style="ticks")

# Create Figure
fig, axs = plt.subplots(1, 2, figsize=(14, 7))

# Accuracy Plot
axs[0].plot(range(1, len(train_acc_history) + 1), train_acc_history, label='Training', linewidth=1.5, color='blue')
axs[0].plot(range(1, len(val_acc_history) + 1), val_acc_history, label='Validation', linewidth=1.5, linestyle='--', color='black')
axs[0].set_xlabel('Epoch', fontsize=16)
axs[0].set_ylabel('Accuracy', fontsize=16)
axs[0].legend(fontsize=17)
axs[0].set_title('')
axs[0].tick_params(axis='both', labelsize=15)  

# Loss Plot
axs[1].plot(range(1, len(train_loss_history) + 1), train_loss_history, label='Training', linewidth=1.5, color='blue')
axs[1].plot(range(1, len(val_loss_history) + 1), val_loss_history, label='Validation', linewidth=1.5, 
            linestyle='--', color='black')
axs[1].set_xlabel('Epoch', fontsize=16)
axs[1].set_ylabel('Loss', fontsize=16)
axs[1].legend(fontsize=17)
axs[1].set_title('')
axs[1].tick_params(axis='both', labelsize=15)  
fig.text(0.29, 0.09, '(a)', fontsize=18, ha='center')
fig.text(0.78, 0.09, '(b)', fontsize=18, ha='center')
plt.tight_layout(rect=[0, 0.1, 1, 1])  
plt.show()

In [ ]:
# Select a Random Index in the Test Set
random_idx = random.randint(0, len(test_loader.dataset) - 1)
sample_image, sample_label = test_loader.dataset[random_idx]
sample_image = sample_image.unsqueeze(0)  # Añadir una dimensión para el batch

# Obtain the Feature Maps
def get_features(name):
    def hook(model, input, output):
        features[name] = output.detach()
    return hook

# Initialize a Dictionary to Store the Feature Maps
features = {}
alexnet.features[0].register_forward_hook(get_features('conv1'))
alexnet.features[3].register_forward_hook(get_features('conv2'))
alexnet.features[6].register_forward_hook(get_features('conv3'))
alexnet.features[8].register_forward_hook(get_features('conv4'))
alexnet.features[10].register_forward_hook(get_features('conv5'))

# Pass the Image Through the Model
alexnet(sample_image)

# Function to Find the Index of the Filter with the Strongest Activations
def get_strongest_activation(feature_map):
    activations = torch.sum(torch.abs(feature_map), dim=[2, 3])
    strongest_filter_idx = torch.argmax(activations, dim=1).item()
    return strongest_filter_idx

# Collect the Activations from Each Convolutional Layer
activations = [sample_image.squeeze(0)]  # Comenzar con la imagen original
conv_layers = ['conv1', 'conv2', 'conv3', 'conv4', 'conv5']
for layer_name in conv_layers:
    strongest_filter_idx = get_strongest_activation(features[layer_name])
    activation_map = features[layer_name][0, strongest_filter_idx]
    activations.append(activation_map)

#Visualize the Activations of Each Convolutional Layer
plt.figure(figsize=(18, 12))

# Limit to a Maximum of 6 Subplots
num_subplots = min(len(activations), 6)
letters = ['(a)', '(b)', '(c)', '(d)', '(e)', '(f)']  

for i in range(num_subplots):
    activation_map = activations[i].cpu().numpy()
    ax = plt.subplot(2, 3, i + 1) 

    if i == 0:
        
        ax.imshow(activation_map.transpose(1, 2, 0), cmap='gray') 
        title_text = 'Input Image'
    else:
        
        ax.imshow(activation_map, cmap='gray')
        title_text = f'Conv. Layer {i}: Random Freature Map'

    
    ax.set_xlabel(f'{letters[i]}', fontsize=18, labelpad=5)  
    ax.set_title('') 
    ax.tick_params(axis='both', labelsize=17)  
    ax.axis('on')

plt.tight_layout()

plt.show()
